# Exploration de données avec Pandas (Exercices)

Ce notebook est inspiré d'une présentation faite par Alexandre Gramfort (INRIA).

In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Objectif: faire une joli visualisation des résultats d'un référendum d'une manière similaire à l'image ci-dessous.
L'autre objectif est de découvrir Pandas en autonomie. Pandas est un module de traitement des données qui permet d'abstraire un grand nombre de formats à travers une abstraction à haut-niveau : le DataFrame.

<img align="left" width=50% src="./image/example_map.png">




## Activités

* Ouvrir le fichier `data/referendum.csv` and présenter les 5 premières lignes pour identifier la structure des données. Le séparateur sera le point virgule.

*Indication*: il existe une méthode de l'object DataFrame qui permet d'afficher les premiers éléments (i.e lignes).

In [ ]:
df_referendum = pd.read_csv("data/referendum.csv", sep=';')

In [ ]:
df_referendum.head()

* Il n'y a pas d'informations sur les régions. On a cependant un deuxième dataset `data/regions.csv` qui contient cette information. Même question qu'au dessus.

In [ ]:
df_regions = pd.read_csv(os.path.join('data', 'regions.csv')) # plus robuste qu'écrire un chemin en toute lettre
df_regions.head()

* Même question pour`data/departments.csv`.

In [ ]:
df_departments = pd.read_csv(os.path.join('data', 'departments.csv'))
df_departments.head()


* Trouvez la colonne dans le dataframe des départements qui est liée à la colonne `code` du dataframe des régions. 
* Fusionnez les deux dataframes en utilisant ces informations. Montrez les 5 premières lignes du dataframe résultant.

*Indication*: il existe une **méthode** pour la fusion de DataFrames.

In [ ]:
# df_referendum a des départments écrits sans le 0. Pour éviter des problèmes d'alignement des données, on nettoie.
df_referendum["Department code"] = df_referendum["Department code"].apply(
    lambda x: "0" + x if len(x) == 1 else x
)

In [ ]:
df_reg_dep = df_departments.merge(df_regions, how='inner', left_on='region_code', right_on='code')
df_reg_dep.head()

* Dans le cadre des données précédentes, une colonne est liée au code du département qui peut être fusionné avec nos données de référendum comme nous l'avons fait précédemment. Puisque nous avons déjà obtenu des informations sur les régions, nous pouvons obtenir le DataFrame des régions avec une nouvelle fusion. 
* Montrez les 5 premières lignes du dataframe fusionné.

In [ ]:
df = df_referendum.merge(df_reg_dep, how='inner', left_on='Department code', right_on='code_x')
df.head()

* Regroupez et additionnez les votes par région et montrez le cadre de données résultant.

*Indication*: c'est une commande bien connue en base de données qui groupe les éléments par....

In [ ]:
regions_vote = df.groupby(['code_y', 'name_y']).sum()
regions_vote

* En prenant l'exemple précédent, tracez le vote pour le "choix A" et le "choix B". Utilisez le fichier `regions.geojson` qui contient des informations géographiques concernant la région. Vous pouvez utiliser `geopandas` qui chargera ces informations dans un dataframe que vous pourrez fusionner avec n'importe quel autre dataframe.
* Une fois les données fusionnées, vous pouvez tracer le nombre absolu et le ratio de votes par région.

In [32]:
!pip install -U geopandas # To install geopandas

In [ ]:

import geopandas as gpd # Il faudra installer ça.

In [ ]:

gdf_regions = gpd.read_file(os.path.join('data', 'regions.geojson'))
gdf_regions = gdf_regions.merge(regions_vote, how='inner', left_on='code', right_on='code_y')

gdf_normalized = gdf_regions.copy()
gdf_normalized['Choice A'] /= gdf_regions[['Choice A', 'Choice B']].sum(axis=1)
gdf_normalized['Choice B'] /= gdf_regions[['Choice A', 'Choice B']].sum(axis=1)

In [ ]:
gdf_normalized.plot(column='Choice A', legend=True, cmap='BuPu')
gdf_normalized.plot(column='Choice B', legend=True, cmap='OrRd')